# Projeto de Arquitetura de Computadores

### Etapa de treino, validação e teste do modelo de DNN (*Deep Neural Networks*)

#### Métricas consideradas:
- LPCs : informações do trato vocal
- CPPS  e Slope : informações do cepstrum (informações que aproximam a análise da percepção acústica humana)

### Aplicação utilizada:
- TensorFlow
  - Arquitetura da rede neural:
    - 3 camadas ocultas, sendo duas com 10 neurônios e uma terceira com 5 neurônios, **relu** como função de ativação;
    - camada de saída com função **sigmoid**
    - 14 características na entrada: 12 coeficientes LPC e as duas medidas cepstrais (CPPS e Slope)



In [1]:
# montando drive para berenciar arquivos
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Importando bibliotecas úteis

In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os

### carregando dataframes de TREINO, VALIDAÇÃO e TESTE

In [25]:
df_TRAIN = pd.read_pickle('/content/drive/MyDrive/proj_arq_2023/train_kay_v1.pkl')
df_TRAIN

,paciente,classe,idx_na_classe,classification,lpc1,lpc2,lpc3,lpc4,lpc5,lpc6,...,minimoF2,maximoF2,rangeF2,mediaF3,desvioF3,minimoF3,maximoF3,rangeF3,slope,cpps
8526,normal36,normal,36,0,-1.796875,0.416748,0.719727,0.298096,-0.775391,-0.076965,...,1836.0,2062.0,225.12500,2760.0,101.062500,2570.0,3012.0,441.25000,8.281250,13.820312
6685,normal30,normal,30,0,-2.410156,1.987305,-0.441650,0.061615,-0.281494,-0.600586,...,1608.0,1789.0,181.12500,2372.0,12.578125,2342.0,2396.0,53.84375,4.867188,11.929688
7399,normal32,normal,32,0,-2.275391,1.236328,1.211914,-1.510742,-0.356445,0.882324,...,1445.0,1845.0,399.25000,2332.0,14.234375,2308.0,2370.0,61.78125,0.688477,17.187500
14607,normal7,normal,7,0,-1.647461,0.148804,0.684570,0.186523,-0.426758,0.147095,...,1268.0,1726.0,458.50000,2172.0,84.812500,1925.0,2360.0,434.50000,0.390137,14.070312
9906,normal40,normal,40,0,-2.218750,1.313477,0.548340,-0.727539,-0.060699,-0.589355,...,1698.0,1754.0,55.90625,2206.0,12.398438,2182.0,2240.0,57.43750,3.406250,13.375000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,edema11,edema,11,1,-1.360352,0.810547,-0.262451,-0.428223,0.967285,-0.915039,...,1848.0,2026.0,177.87500,2754.0,64.312500,2568.0,2854.0,287.50000,0.895508,5.035156
9327,paralisia24,paralisia,24,1,-0.732422,0.838379,-1.094727,0.804688,-1.241211,0.821777,...,1886.0,2840.0,954.50000,2974.0,344.250000,2578.0,3930.0,1352.00000,2.615234,4.296875
6680,nodulo3,nodulo,3,1,-1.272461,-0.531738,0.990723,0.165405,-0.652832,0.156250,...,1324.0,1533.0,209.12500,2382.0,86.187500,2250.0,2580.0,330.50000,-8.062500,11.757812
2749,edema30,edema,30,1,-1.134766,-0.104675,-0.384521,0.565430,0.086182,0.395020,...,1051.0,1786.0,735.00000,2378.0,232.875000,1759.0,2726.0,968.00000,-6.996094,12.687500


In [27]:
# contando quantos casos por clase
contagem_TRAIN = df_TRAIN['classification'].value_counts()
contagem_TRAIN
#df_TRAIN.groupby("classification").count()

0    10635
1     9373
Name: classification, dtype: int64

In [28]:
df_VALID = pd.read_pickle('/content/drive/MyDrive/proj_arq_2023/valid_kay_v1.pkl')
df_VALID

,paciente,classe,idx_na_classe,classification,lpc1,lpc2,lpc3,lpc4,lpc5,lpc6,...,minimoF2,maximoF2,rangeF2,mediaF3,desvioF3,minimoF3,maximoF3,rangeF3,slope,cpps
4776,normal24,normal,24,0,-2.027344,1.218750,0.342285,-0.970703,1.268555,-1.385742,...,1574.0,2646.0,1072.000,2652.0,274.500000,2396.0,3856.0,1460.0,7.148438,14.656250
7997,normal34,normal,34,0,-2.259766,1.353516,0.455322,-0.944336,0.828125,-0.691895,...,938.5,2200.0,1261.000,2880.0,577.000000,1784.0,4700.0,2914.0,-7.972656,16.953125
4276,normal23,normal,23,0,-2.009766,0.941406,0.520996,-0.154419,-0.557617,0.246216,...,1635.0,2116.0,480.500,2652.0,151.875000,2438.0,3140.0,700.5,6.957031,13.078125
14812,normal8,normal,8,0,-1.876953,1.071289,-0.115662,-0.336182,0.623047,-0.677734,...,1744.0,2120.0,376.250,2620.0,121.937500,2056.0,2812.0,756.0,0.012909,18.062500
10780,normal43,normal,43,0,-1.947266,1.073242,-0.019073,0.221436,-0.100891,-0.851074,...,1724.0,1871.0,146.625,1988.0,84.562500,1844.0,2256.0,412.5,-4.574219,21.734375
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5510,nodulo12,nodulo,12,1,-1.715820,1.736328,-2.453125,2.335938,-2.326172,2.720703,...,1062.0,1249.0,186.375,2674.0,112.875000,2388.0,3190.0,803.0,-15.492188,14.382812
5330,nodulo10,nodulo,10,1,-1.468750,1.164062,-1.381836,0.713379,-0.831543,1.682617,...,785.5,2742.0,1956.000,2902.0,255.875000,2078.0,3542.0,1464.0,-20.250000,9.125000
4531,edema44,edema,44,1,-1.239258,-0.338623,0.265869,0.765137,-0.779297,0.446289,...,1702.0,1910.0,207.875,2716.0,187.625000,2304.0,3318.0,1012.5,-22.375000,11.289062
9866,paralisia28,paralisia,28,1,-0.983887,0.281006,-0.828125,0.510742,-0.529297,1.038086,...,1059.0,1226.0,166.875,2642.0,49.125000,2522.0,2794.0,272.5,-17.937500,9.476562


In [29]:
contagem_VALID = df_VALID['classification'].value_counts()
contagem_VALID
#df_VALID.groupby("classification").count()

0    1519
1    1339
Name: classification, dtype: int64

In [30]:
df_TEST = pd.read_pickle('/content/drive/MyDrive/proj_arq_2023/test_kay_v1.pkl')
df_TEST

,paciente,classe,idx_na_classe,classification,lpc1,lpc2,lpc3,lpc4,lpc5,lpc6,...,minimoF2,maximoF2,rangeF2,mediaF3,desvioF3,minimoF3,maximoF3,rangeF3,slope,cpps
3456,normal20,normal,20,0,-2.722656,2.458984,-0.347412,0.040131,-1.498047,1.240234,...,1603.0,1830.0,227.3750,2578.0,27.015625,2512.0,2726.0,213.875,7.191406,13.367188
11768,normal47,normal,47,0,-2.320312,1.642578,0.280518,-0.875977,0.140747,0.436279,...,1099.0,2524.0,1425.0000,2456.0,347.000000,2017.0,3370.0,1353.000,4.253906,12.109375
3271,normal2,normal,2,0,-2.316406,1.840820,0.163452,-1.379883,0.697754,0.068970,...,1269.0,2034.0,765.5000,2070.0,48.500000,1937.0,2200.0,263.500,0.663086,17.562500
4940,normal25,normal,25,0,-2.191406,1.541992,-0.390869,0.258301,0.239136,-1.196289,...,1366.0,1988.0,622.5000,1923.0,38.187500,1894.0,2338.0,444.750,-2.378906,20.015625
9223,normal39,normal,39,0,-2.275391,1.563477,-0.027924,0.013855,-0.475342,0.074402,...,1978.0,2068.0,91.3125,2600.0,167.125000,2138.0,3040.0,903.000,0.963379,21.359375
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11964,paralisia44,paralisia,44,1,-1.057617,0.106018,-0.521484,0.584473,-0.058990,0.274658,...,1567.0,2250.0,683.5000,2362.0,145.375000,2092.0,3062.0,971.000,-7.253906,6.878906
5191,nodulo1,nodulo,1,1,-1.823242,0.687012,0.406006,-0.249512,-0.746094,1.187500,...,1544.0,1670.0,126.3125,2414.0,125.562500,1644.0,2540.0,897.000,-2.830078,11.921875
5390,nodulo10,nodulo,10,1,-1.194336,0.646973,-0.992188,0.319336,-0.704590,1.610352,...,785.5,2742.0,1956.0000,2902.0,255.875000,2078.0,3542.0,1464.000,-20.250000,9.125000
860,edema16,edema,16,1,-1.297852,-0.236572,0.184814,0.707031,-0.241211,-0.044098,...,1566.0,1691.0,125.3125,2704.0,68.437500,2524.0,2872.0,348.000,-11.390625,10.796875


In [31]:
contagem_TEST = df_TEST['classification'].value_counts()
contagem_TEST
#df_TEST.groupby("classification").count()

0    3039
1    2679
Name: classification, dtype: int64

### gerenciando dados de TREINO, VALIDAÇÃO  e TESTE

 - escolhendo as features utilizadas
 - selecionando a coluna TARGET

In [32]:
df_TRAIN.keys()

Index(['paciente', 'classe', 'idx_na_classe', 'classification', 'lpc1', 'lpc2',
       'lpc3', 'lpc4', 'lpc5', 'lpc6', 'lpc7', 'lpc8', 'lpc9', 'lpc10',
       'lpc11', 'lpc12', 'lpc13', 'lpc14', 'lpc15', 'lpc16', 'lpc17', 'lpc18',
       'lpc19', 'lpc20', 'lpc21', 'lpc22', 'lpc23', 'lpc24', 'lpc25', 'lpc26',
       'lpc27', 'lpc28', 'lpc29', 'mfcc1', 'mfcc2', 'mfcc3', 'mfcc4', 'mfcc5',
       'mfcc6', 'mfcc7', 'mfcc8', 'mfcc9', 'mfcc10', 'mfcc11', 'mfcc12',
       'dmfcc1', 'dmfcc2', 'dmfcc3', 'dmfcc4', 'dmfcc5', 'dmfcc6', 'dmfcc7',
       'dmfcc8', 'dmfcc9', 'dmfcc10', 'dmfcc11', 'dmfcc12', 'ddmfcc1',
       'ddmfcc2', 'ddmfcc3', 'ddmfcc4', 'ddmfcc5', 'ddmfcc6', 'ddmfcc7',
       'ddmfcc8', 'ddmfcc9', 'ddmfcc10', 'ddmfcc11', 'ddmfcc12', 'mediaF0',
       'desvioFO', 'minimoF0', 'maximoF0', 'rangeF0', 'jitter', 'shimmer',
       'hnr', 'gne', 'mediaF1', 'desvioF1', 'minimoF1', 'maximoF1', 'rangeF1',
       'mediaF2', 'desvioF2', 'minimoF2', 'maximoF2', 'rangeF2', 'mediaF3',
       'des

In [33]:
# Dos dados de TREINO
target_TRAIN = df_TRAIN.pop('classification')
numeric_feature_names_TRAIN = ['cpps', 'slope','lpc1',  'lpc2',
                         'lpc3', 'lpc4', 'lpc5', 'lpc6',
                         'lpc7', 'lpc8','lpc9', 'lpc10',
                         'lpc11', 'lpc12']
numeric_features_TRAIN = df_TRAIN[numeric_feature_names_TRAIN]
numeric_features_TRAIN.head()

,cpps,slope,lpc1,lpc2,lpc3,lpc4,lpc5,lpc6,lpc7,lpc8,lpc9,lpc10,lpc11,lpc12
8526,13.820312,8.281250,-1.796875,0.416748,0.719727,0.298096,-0.775391,-0.076965,-0.407227,1.616211,-0.676270,-0.539062,-0.071533,0.600586
6685,11.929688,4.867188,-2.410156,1.987305,-0.441650,0.061615,-0.281494,-0.600586,1.128906,0.167969,-0.724609,-0.258545,0.470459,0.262939
7399,17.187500,0.688477,-2.275391,1.236328,1.211914,-1.510742,-0.356445,0.882324,0.473145,-0.973633,0.066956,0.772949,-0.657227,0.118225
14607,14.070312,0.390137,-1.647461,0.148804,0.684570,0.186523,-0.426758,0.147095,-0.299561,0.304932,-0.062164,0.256348,-0.621582,0.532715
9906,13.375000,3.406250,-2.218750,1.313477,0.548340,-0.727539,-0.060699,-0.589355,1.739258,-0.814453,-0.392334,-0.475342,1.477539,-0.403076


In [34]:
# Dos dados de VALIDAÇÃO
target_VALID = df_VALID.pop('classification')
numeric_feature_names_VALID = ['cpps', 'slope','lpc1',  'lpc2',
                         'lpc3', 'lpc4', 'lpc5', 'lpc6',
                         'lpc7', 'lpc8','lpc9', 'lpc10',
                         'lpc11', 'lpc12']
numeric_features_VALID = df_VALID[numeric_feature_names_VALID]
numeric_features_VALID.head()

,cpps,slope,lpc1,lpc2,lpc3,lpc4,lpc5,lpc6,lpc7,lpc8,lpc9,lpc10,lpc11,lpc12
4776,14.656250,7.148438,-2.027344,1.218750,0.342285,-0.970703,1.268555,-1.385742,0.288574,0.890625,-0.278320,-0.335693,-0.086548,-0.019974
7997,16.953125,-7.972656,-2.259766,1.353516,0.455322,-0.944336,0.828125,-0.691895,0.147827,-0.006214,0.260254,0.306396,-0.746094,0.459473
4276,13.078125,6.957031,-2.009766,0.941406,0.520996,-0.154419,-0.557617,0.246216,-0.065857,0.635742,-0.521973,-0.230835,-0.039001,0.834961
14812,18.062500,0.012909,-1.876953,1.071289,-0.115662,-0.336182,0.623047,-0.677734,0.730469,-0.542480,-0.055634,0.502930,-0.136475,-0.273926
10780,21.734375,-4.574219,-1.947266,1.073242,-0.019073,0.221436,-0.100891,-0.851074,0.528320,-0.024399,0.661621,-0.122253,-0.853027,0.093872


In [35]:
# Dos dados de TESTE
target_TEST = df_TEST.pop('classification')
numeric_feature_names_TEST = ['cpps', 'slope','lpc1',  'lpc2',
                         'lpc3', 'lpc4', 'lpc5', 'lpc6',
                         'lpc7', 'lpc8','lpc9', 'lpc10',
                         'lpc11', 'lpc12']
numeric_features_TEST = df_TEST[numeric_feature_names_TEST]
numeric_features_TEST.head()

,cpps,slope,lpc1,lpc2,lpc3,lpc4,lpc5,lpc6,lpc7,lpc8,lpc9,lpc10,lpc11,lpc12
3456,13.367188,7.191406,-2.722656,2.458984,-0.347412,0.040131,-1.498047,1.240234,-0.038055,0.745605,-1.336914,-0.095825,0.870117,0.253662
11768,12.109375,4.253906,-2.320312,1.642578,0.280518,-0.875977,0.140747,0.436279,-0.588867,0.613281,-0.188965,-0.683105,0.895508,0.197021
3271,17.562500,0.663086,-2.316406,1.840820,0.163452,-1.379883,0.697754,0.068970,0.324951,-0.729492,0.303711,0.227417,0.123474,-0.520508
4940,20.015625,-2.378906,-2.191406,1.541992,-0.390869,0.258301,0.239136,-1.196289,0.359131,0.863770,-0.346436,0.377197,-0.892578,0.114746
9223,21.359375,0.963379,-2.275391,1.563477,-0.027924,0.013855,-0.475342,0.074402,-0.057434,0.639648,-0.359619,-0.462646,0.615234,-0.286133


### Aplicando TensorFlow

In [36]:
# Função auxiliar para arquitetura
def get_compiled_model():
  model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(5, activation='relu'),
    tf.keras.layers.Dense(1)
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                metrics=['accuracy'])
  return model

#### Treinamento

In [38]:
# convertendo features para TENSOR
tf.convert_to_tensor(numeric_features_TRAIN)

<tf.Tensor: shape=(20008, 14), dtype=float16, numpy=
array([[ 13.82   ,   8.28   ,  -1.797  , ...,  -0.539  ,  -0.07153,
          0.6006 ],
       [ 11.93   ,   4.867  ,  -2.41   , ...,  -0.2585 ,   0.4705 ,
          0.263  ],
       [ 17.19   ,   0.6885 ,  -2.275  , ...,   0.773  ,  -0.657  ,
          0.1182 ],
       ...,
       [ 11.76   ,  -8.06   ,  -1.272  , ...,   0.6313 ,   0.1747 ,
         -0.622  ],
       [ 12.69   ,  -6.996  ,  -1.135  , ...,   0.2449 ,  -0.0694 ,
          0.352  ],
       [ 11.98   , -25.06   ,  -1.338  , ...,   0.5146 ,  -0.6475 ,
          0.9336 ]], dtype=float16)>

In [40]:
# Aplicando normalização das features (para equalizar grandezas)
normalizer_TRAIN = tf.keras.layers.Normalization(axis=-1)
normalizer_TRAIN.adapt(numeric_features_TRAIN)

In [41]:
# verificando normalização
normalizer_TRAIN(numeric_features_TRAIN.iloc[:3])

<tf.Tensor: shape=(3, 14), dtype=float32, numpy=
array([[ 0.11936525,  1.5106283 , -0.24510084, -0.38331285,  0.83783114,
         0.23596363, -0.63063884, -0.4584674 , -0.28463653,  1.6522307 ,
        -0.5051588 , -1.0893012 ,  0.29803097,  0.5455026 ],
       [-0.34613642,  1.1116415 , -1.5814666 ,  1.6620228 , -0.50981545,
        -0.06477748,  0.06265794, -1.1034055 ,  1.8120884 , -0.23329514,
        -0.57328594, -0.7078235 ,  1.0806637 ,  0.01652278],
       [ 0.9484198 ,  0.6232934 , -1.2878065 ,  0.68402624,  1.408959  ,
        -2.0644011 , -0.04255318,  0.72307956,  0.9170147 , -1.7195932 ,
         0.54229605,  0.694912  , -0.5477058 , -0.21019654]],
      dtype=float32)>

In [42]:
# Realizando o TREINAMENTO com a arquitetura
model = get_compiled_model()
model.fit(numeric_features_TRAIN, target_TRAIN, epochs=50, batch_size=64)

Epoch 1/50
313/313 [==============================] - 2s 2ms/step - loss: 0.3447 - accuracy: 0.7801
Epoch 2/50
313/313 [==============================] - 1s 2ms/step - loss: 0.1935 - accuracy: 0.9180
Epoch 3/50
313/313 [==============================] - 1s 2ms/step - loss: 0.1456 - accuracy: 0.9354
Epoch 4/50
313/313 [==============================] - 1s 2ms/step - loss: 0.1083 - accuracy: 0.9519
Epoch 5/50
313/313 [==============================] - 1s 2ms/step - loss: 0.0862 - accuracy: 0.9621
Epoch 6/50
313/313 [==============================] - 1s 2ms/step - loss: 0.0748 - accuracy: 0.9681
Epoch 7/50
313/313 [==============================] - 1s 2ms/step - loss: 0.0666 - accuracy: 0.9726
Epoch 8/50
313/313 [==============================] - 1s 2ms/step - loss: 0.0604 - accuracy: 0.9748
Epoch 9/50
313/313 [==============================] - 1s 2ms/step - loss: 0.0539 - accuracy: 0.9775
Epoch 10/50
313/313 [==============================] - 1s 2ms/step - loss: 0.0477 - accuracy: 0.9808

In [43]:
# Detalhe da arquitetura treinada
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                150       
                                                                 
 dense_1 (Dense)             (None, 10)                110       
                                                                 
 dense_2 (Dense)             (None, 5)                 55        
                                                                 
 dense_3 (Dense)             (None, 1)                 6         
                                                                 
Total params: 321
Trainable params: 321
Non-trainable params: 0
_________________________________________________________________


#### VALIDAÇÃO

In [44]:
# convertendo features para TENSOR
tf.convert_to_tensor(numeric_features_VALID)

<tf.Tensor: shape=(2858, 14), dtype=float16, numpy=
array([[ 1.466e+01,  7.148e+00, -2.027e+00, ..., -3.357e-01, -8.655e-02,
        -1.997e-02],
       [ 1.695e+01, -7.973e+00, -2.260e+00, ...,  3.064e-01, -7.461e-01,
         4.595e-01],
       [ 1.308e+01,  6.957e+00, -2.010e+00, ..., -2.308e-01, -3.900e-02,
         8.350e-01],
       ...,
       [ 1.129e+01, -2.238e+01, -1.239e+00, ...,  3.867e-01, -4.336e-01,
         5.674e-01],
       [ 9.477e+00, -1.794e+01, -9.839e-01, ...,  9.097e-01, -9.355e-01,
         6.997e-01],
       [ 1.352e+01, -2.308e+01, -1.135e+00, ...,  5.295e-02, -6.060e-01,
         9.609e-01]], dtype=float16)>

In [45]:
# Aplicando normalização das features (para equalizar grandezas)
normalizer_VALID = tf.keras.layers.Normalization(axis=-1)
normalizer_VALID.adapt(numeric_features_VALID)

In [46]:
# verificando normalização
normalizer_VALID(numeric_features_VALID.iloc[:3])

<tf.Tensor: shape=(3, 14), dtype=float32, numpy=
array([[ 0.3151667 ,  1.4007207 , -0.7498504 ,  0.6524397 ,  0.41562387,
        -1.4161495 ,  2.2718592 , -2.1115136 ,  0.69508874,  0.69824946,
         0.05235213, -0.7998117 ,  0.28011322, -0.4218538 ],
       [ 0.8776232 , -0.40377262, -1.260651  ,  0.8276693 ,  0.54794574,
        -1.3824502 ,  1.6504983 , -1.237355  ,  0.4998727 , -0.49620748,
         0.80096745,  0.06195871, -0.6662865 ,  0.31136075],
       [-0.07128301,  1.377879  , -0.7112184 ,  0.29182225,  0.6248238 ,
        -0.37287813, -0.30451542, -0.05545686,  0.2034931 ,  0.35878327,
        -0.2863233 , -0.6590777 ,  0.34833878,  0.88559276]],
      dtype=float32)>

In [47]:
# Realizando a VALIDAÇÃO com a arquitetura
model.evaluate(numeric_features_VALID, target_VALID, verbose=2)

90/90 - 0s - loss: 0.0157 - accuracy: 0.9958 - 409ms/epoch - 5ms/step


[0.01569109410047531, 0.9958012700080872]

#### TESTE

In [48]:
# convertendo features para TENSOR
tf.convert_to_tensor(numeric_features_TEST)

<tf.Tensor: shape=(5718, 14), dtype=float16, numpy=
array([[ 1.337e+01,  7.191e+00, -2.723e+00, ..., -9.583e-02,  8.701e-01,
         2.537e-01],
       [ 1.211e+01,  4.254e+00, -2.320e+00, ..., -6.831e-01,  8.955e-01,
         1.970e-01],
       [ 1.756e+01,  6.631e-01, -2.316e+00, ...,  2.274e-01,  1.235e-01,
        -5.205e-01],
       ...,
       [ 9.125e+00, -2.025e+01, -1.194e+00, ...,  9.800e-01, -1.409e+00,
         1.406e+00],
       [ 1.080e+01, -1.139e+01, -1.298e+00, ..., -1.032e-01,  3.003e-01,
        -5.117e-01],
       [ 8.922e+00, -7.984e+00, -1.124e+00, ..., -1.772e-01, -9.560e-03,
         1.714e-01]], dtype=float16)>

In [49]:
# Aplicando normalização das features (para equalizar grandezas)
normalizer_TEST = tf.keras.layers.Normalization(axis=-1)
normalizer_TEST.adapt(numeric_features_TEST)

In [50]:
# verificando normalização
normalizer_TEST(numeric_features_TEST.iloc[:3])

<tf.Tensor: shape=(3, 14), dtype=float32, numpy=
array([[ 0.00457244,  1.4025059 , -2.262316  ,  2.2936947 , -0.41157356,
        -0.07833334, -1.6410644 ,  1.1559807 ,  0.23743592,  0.5126696 ,
        -1.441317  , -0.48046887,  1.6477727 , -0.00248993],
       [-0.30494565,  1.0586089 , -1.3857118 ,  1.2239923 ,  0.32077754,
        -1.2406087 ,  0.6536735 ,  0.15926646, -0.507387  ,  0.34002975,
         0.17668271, -1.2756735 ,  1.6843647 , -0.09086177],
       [ 1.0369402 ,  0.6382269 , -1.3772011 ,  1.4837407 ,  0.18424474,
        -1.8799202 ,  1.433628  , -0.2961106 ,  0.7283035 , -1.411851  ,
         0.87109435, -0.04278399,  0.57173955, -1.2103653 ]],
      dtype=float32)>

In [51]:
# Realizando o TESTE com a arquitetura
model.evaluate(numeric_features_TEST, target_TEST, verbose=2)

179/179 - 0s - loss: 0.0143 - accuracy: 0.9955 - 419ms/epoch - 2ms/step


[0.014304318465292454, 0.9954529404640198]

## Modelo PRONTO

#### salvando o modelo para embarcar depois

In [52]:
caminho_modelo = '/content/drive/MyDrive/proj_arq_2023/'
caminho_modelo

'/content/drive/MyDrive/proj_arq_2023/'

In [53]:
model.save(caminho_modelo+'project_model.h5')

In [58]:
# checando o diretório
os.listdir(caminho_modelo)

['dataset_medias_kay_v1.pkl',
 'dataset_matrix_sdl_kay_v2.pkl',
 'dataset_matrix_ptl_kay_v2.pkl',
 'dataset_matrix_sdl_kay_v3.pkl',
 'dataset_matrix_ptl_kay_v3.pkl',
 'train_kay_v1.pkl',
 'valid_kay_v1.pkl',
 'test_kay_v1.pkl',
 'project_model.h5']

In [63]:
# Lista todos os arquivos no diretório especificado
arquivos = os.listdir(caminho_modelo)

# Itera sobre cada arquivo e obtém o tamanho em memória
for arquivo in arquivos:
    caminho_arquivo = os.path.join(caminho_modelo, arquivo)
    tamanho_bytes = os.path.getsize(caminho_arquivo)
    tamanho_mb = tamanho_bytes / (1024 * 1024)  # Convertendo para MB
    print(f"Tamanho do arquivo {arquivo}: {tamanho_mb:.3f} MB")

Tamanho do arquivo dataset_medias_kay_v1.pkl: 0.144 MB
Tamanho do arquivo dataset_matrix_sdl_kay_v2.pkl: 10.840 MB
Tamanho do arquivo dataset_matrix_ptl_kay_v2.pkl: 9.555 MB
Tamanho do arquivo dataset_matrix_sdl_kay_v3.pkl: 2.755 MB
Tamanho do arquivo dataset_matrix_ptl_kay_v3.pkl: 2.429 MB
Tamanho do arquivo train_kay_v1.pkl: 3.781 MB
Tamanho do arquivo valid_kay_v1.pkl: 0.543 MB
Tamanho do arquivo test_kay_v1.pkl: 1.083 MB
Tamanho do arquivo project_model.h5: 0.038 MB


## Carregando o modelo salvo e testando

In [64]:
# Carregue o modelo
modelo_carregado = tf.keras.models.load_model(caminho_modelo+'project_model.h5')


In [65]:
modelo_carregado.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                150       
                                                                 
 dense_1 (Dense)             (None, 10)                110       
                                                                 
 dense_2 (Dense)             (None, 5)                 55        
                                                                 
 dense_3 (Dense)             (None, 1)                 6         
                                                                 
Total params: 321
Trainable params: 321
Non-trainable params: 0
_________________________________________________________________


In [68]:
# avaliando o TESTE novamente
modelo_carregado.evaluate(numeric_features_TEST, target_TEST, verbose=2)

179/179 - 0s - loss: 0.0143 - accuracy: 0.9955 - 367ms/epoch - 2ms/step


[0.014304318465292454, 0.9954529404640198]

In [70]:
# Testando como seria a PREDIÇÃO DA CLASSE
predictions = modelo_carregado.predict(numeric_features_TEST)
predicted_classes = (predictions > 0.5).astype(int)
print(predicted_classes)

print(len(predicted_classes == 1))
print(len(predicted_classes == 0))

179/179 [==============================] - 0s 1ms/step
[[0]
 [0]
 [0]
 ...
 [1]
 [1]
 [1]]
5718
5718


In [72]:
np.shape(predicted_classes)

(5718, 1)

In [73]:
np.reshape(predicted_classes, len(predicted_classes))

array([0, 0, 0, ..., 1, 1, 1])

In [ ]:
def get_prob(predictions_vector):
  '''
  Esta função pega as predições e retorna a probabilidade
  de o paciente ter uma condição patológica
  '''
  pred_vec = np.reshape(predictions_vector, len(predictions_vector))
  contagem = np.bincount(pred_vec)

  prob_sdl = ( contagem[0] / len(pred_vec)) * 100
  prob_ptl = ( contagem[1] / len(pred_vec)) * 100

  return prob_sdl , prob_ptl


In [76]:
sdl_p , ptl_p = get_prob(predicted_classes)
print(f"Probabilidade de ter patologia: {ptl_p:.2f} %")
print(f"Probabilidade de ter saúde: {sdl_p:.2f} %")

Probabilidade de ter patologia: 46.50 %
Probabilidade de ter saúde: 53.50 %
